```
사진에 있는 선반과 물체들을 감지하는 기능
ㄴ 선반을 사직으로 찍게 할건지 선반의 종류를 사용자가 선택하게 할건지
감지된 물체중에 사격형이 아닌 물체를 사각화 시켜주는 기능

사각화 된 물체를 종류에 맞게 분리해주는 기능

분리된 물체를 선반에 맞춰 정리해주는기능

정리된 모습의 물체화 선반을 단말기에 출력하는 기능
```

In [33]:
from ultralytics import YOLO
from ultralytics.engine.results import Results

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x-seg.pt')

# Run inference on an image
results: Results = model('20240502_142118.jpg')  # results list

100%|██████████| 137M/137M [00:02<00:00, 59.2MB/s] 



image 1/1 l:\2024-1\SW\0429   \Arrangement_of_Jinn_Dolsoe\20240502_142118.jpg: 384x640 3 persons, 1 backpack, 2 bottles, 2 chairs, 2 laptops, 1 mouse, 1 remote, 1 keyboard, 1 cell phone, 457.5ms
Speed: 2.0ms preprocess, 457.5ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)


In [34]:
class MyBoxes:
    def __init__(self, object, xyxy, conf):
        self.object = object
        self.xyxy = xyxy
        self.conf = conf

    def show(self):
        print(f"xyxy: {self.xyxy}")
        print(f"conf: {self.conf}")

In [36]:
boxes = list()

# View results
for r in results[0]:
    boxes.append(MyBoxes(r, r.boxes.xyxy, r.boxes.conf))
    # print(r.boxes)  # print the Boxes object containing the detection bounding boxes
    # print(r.boxes.xyxy)  # print the xyxy tensor of bounding boxes
    # print(r.boxes.conf)  # print the confidence tensor of bounding boxes

results[0].show()
# boxes[7].show()
boxes[7].object.show()
print(boxes[7].object.masks.xy)

arr = boxes[7].object.masks.xy

[array([[     1006.2,         401],
       [       1000,      407.25],
       [     993.75,      407.25],
       [      987.5,       413.5],
       [     981.25,       413.5],
       [        975,      419.75],
       [     956.25,      419.75],
       [        950,         426],
       [     893.75,         426],
       [      887.5,      432.25],
       [     881.25,      432.25],
       [        875,       438.5],
       [     868.75,       438.5],
       [      862.5,      444.75],
       [     856.25,      444.75],
       [        850,         451],
       [     831.25,         451],
       [        825,      457.25],
       [        825,       488.5],
       [     856.25,      519.75],
       [        925,      519.75],
       [     931.25,       513.5],
       [     943.75,       513.5],
       [     956.25,         501],
       [     968.75,         501],
       [        975,      494.75],
       [     1056.2,      494.75],
       [     1062.5,       488.5],
       [       1075

imread로 원래 있던 그림을 기본으로 하고 float32형 array배열의 arr안에 있는 데이터를 점으로 cv2로 찍고싶어

In [38]:
import cv2

# Read the original image using imread
image = cv2.imread("20240502_142118.jpg")

# Iterate over the data points in arr and draw them as points on the image
for point in arr[0]:
    print(point)
    x, y = point
    cv2.circle(image, (int(x), int(y)), radius=2, color=(0, 0, 255), thickness=-1)

# Display the image with the plotted points
cv2.imshow("Image with Points", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[     1006.2         401]
[       1000      407.25]
[     993.75      407.25]
[      987.5       413.5]
[     981.25       413.5]
[        975      419.75]
[     956.25      419.75]
[        950         426]
[     893.75         426]
[      887.5      432.25]
[     881.25      432.25]
[        875       438.5]
[     868.75       438.5]
[      862.5      444.75]
[     856.25      444.75]
[        850         451]
[     831.25         451]
[        825      457.25]
[        825       488.5]
[     856.25      519.75]
[        925      519.75]
[     931.25       513.5]
[     943.75       513.5]
[     956.25         501]
[     968.75         501]
[        975      494.75]
[     1056.2      494.75]
[     1062.5       488.5]
[       1075       488.5]
[     1087.5         476]
[       1100         476]
[     1106.2      469.75]
[       1175      469.75]
[     1181.2       463.5]
[     1193.8       463.5]
[     1206.2         451]
[     1218.8         451]
[       1225      444.75]
[       1250